In [1]:
import ms4_franklab_pyplines as pyp
import pyff_utils as pyff
import ms4_franklab_proc2py as p2p
import sys, os, subprocess
import logging
from distutils.dir_util import copy_tree
%load_ext autoreload

In [2]:
%autoreload 2

In [13]:
# anim details. for now, assume that sort should include all eps/day
mda_util_path = '/home/anna/Src/MLpackages/franklab_MS4/mda_util.py'
ntlinks_path = '/home/anna/Src/MLpackages/franklab_MS4/MS4setup_NTlinks.node.js'
data_dir = '/mnt/vortex/data1_backup/anna/'
results_dir='/media/anna/whirlwindtemp1/'
anim = 'gus'
days = [1]
dates = [20171022, 20171023, 20171024]

# if datelist not provided, fetch dates from taskinfo struct
# TODO make py_evaluate filter more flexible so that it can do this

#if not dates or len(days)!=len(dates):
#    dates = pyff.py_evaluatefilter(base_dir, anim, 'taskinfo', days,['date','*'])

#if tetlist not provided, fetch ca1 tets from tetinfo struct
tetfilt = ['area','ca1']
tetlist = []
if not tetlist and tetfilt:
    tetlist = pyff.py_evaluatefilter(results_dir, anim, 'tetinfo', days,tetfilt)

In [14]:
tetlist

defaultdict(list, {1: [1, 22, 23, 29]})

In [20]:
for index, day in enumerate(days):

    day_src_dir = data_dir+anim+'/preprocessing/'+str(dates[index])+'/'
    # check if mnt directory exists for this day. if not, call mda_util to generate the symlinks to the mda files
    # run this in the data src directory (vortex)
    mnt_path = day_src_dir+str(dates[index])+'_'+anim+'.mnt/'
    print('processing day '+str(day))
    if not os.path.exists(mnt_path):
        print('no mnt directory found; calling mda_util')
        os.chdir(data_dir+anim+'/preprocessing')
        subprocess.call(['python',mda_util_path])  
    
    # check if .mountain dir exists for this day (still in the data src  dir)
    # if not, run the setup javascript to generate prvs that link mda files (via their symlinks) across epochs
    # need to generate mountaindir in src then copy it over to results
    mountain_src_path = day_src_dir+str(dates[index])+'_'+anim+'.mountain'
    mountain_res_path = results_dir+anim+'/preprocessing/'+str(dates[index])+'/'+str(dates[index])+'_'+anim+'.mountain'
    if not os.path.exists(mountain_src_path):
        print('no mountain dir found; calling setup_NTlinks')
        os.chdir(day_src_dir)
        subprocess.call([ntlinks_path, mnt_path])
    if not os.path.exists(mountain_res_path):
        print('copying mountain dir to results dir')
        copy_tree(mountain_src_path,mountain_res_path)
    
    #now for the actually processing: iterate through ntrodes
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_src_dir = mountain_src_path+'/nt'+str(nt)
        nt_out_dir = mountain_res_path+'/nt'+str(nt)
        
        # concatenate all eps, since ms4 no longer takes a list of mdas; save as raw.mda
        # save this to the output dir; it serves as src for subsequent steps
        prv_list=nt_src_dir+'/raw.mda.prv'
        pyp.concat_eps(dataset_dir=nt_out_dir,mda_list=prv_list)
        
        # preprocessing: filter, mask out artifacts, whiten
        # TODO make optional whether you save the interediates (filt, pre)
        pyp.filt_mask_whiten(dataset_dir=nt_out_dir,output_dir=nt_out_dir, freq_min=300,freq_max=6000, opts={})
        
        #run the actual sort 
        pyp.ms4_sort_on_segs(dataset_dir=nt_out_dir,output_dir=nt_out_dir, adjacency_radius=-1,detect_threshold=3, detect_sign=-1, opts={})
        
        

processing day 1
RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ms3.mask_out_artifacts --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv --parameters interval_size:2000 threshold:5 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and sub

RUNNING: ml-run-process pyms.extract_timeseries --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv --parameters t1:47461993 t2:156381991 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre-2.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/bin/python3 /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ml_pyms/basic/basic.py pyms.extract_timeseries --timeseries=/media/anna/whirlwindtemp2/MSjstmp/output_3f0e7d4ebd22fdff82afbead8d07468b02fe9074_timeseries_out.mda --channels_array= --timeseries_out=/med

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/bin/python3 /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --timeseries=/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre-3.mda --geom= --firings_out=/media/anna/whirlwindtemp2/MSjstmp/tempdir_b91e051f89_HlOGqy/output_firings_out.mda --adjacency_radius=-1 --detect_sign=-1 --detect_threshold=3 --_tempdir=/media/anna/whirlwindtemp2/MSjstmp/tempdir_b91e051f89_HlOGqy
Using tempdir=/media/anna/whirlwindtemp2/MSjstmp/tempdir_b91e051f89_HlOGqy
Preparing /media/anna/whirlwindtemp2/MSjstmp/tempdir_b91e051f89_HlOGqy/timeseries.hdf5...
Preparing neighborhood sorters...

Detecting events on channel 1 (phase1)

RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/firings_raw.mda timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv --parameters samplerate:30000 --outputs cluster_metrics_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_c42348872c10b3e0238286a657e228df25306aaa.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ms3.mp ms3.cluster_metrics --firings=/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gu

RUNNING: ml-run-process pyms.extract_timeseries --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/pre.mda.prv --parameters t1:0 t2:47461992 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/pre-1.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/bin/python3 /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ml_pyms/basic/basic.py pyms.extract_timeseries --timeseries=/media/anna/whirlwindtemp2/MSjstmp/output_4cf2655c9bcbc9392251f7f0e3e65f80936587ea_timeseries_out.mda --channels_array= --timeseries_out=/media/an

...

RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/firings_raw.mda timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre.mda.prv --parameters samplerate:30000 --outputs cluster_metrics_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_68b82cb55cea455d7a7951b124df5ded4ad895cb.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ms3.mp ms3.cluster_metrics --firings=/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022

RUNNING: ml-run-process pyms.extract_timeseries --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre.mda.prv --parameters t1:0 t2:47461992 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre-1.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/bin/python3 /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ml_pyms/basic/basic.py pyms.extract_timeseries --timeseries=/media/anna/whirlwindtemp2/MSjstmp/output_4fcdf8afb6b16653c99e6c9c724f77a92170946a_timeseries_out.mda --channels_array= --timeseries_out=/media/an

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /home/anna/anaconda3/envs/ms4/bin/python3 /home/anna/anaconda3/envs/ms4/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --timeseries=/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre-2.mda --geom= --firings_out=/media/anna/whirlwindtemp2/MSjstmp/tempdir_9cacaa7f0f_IqVfQp/output_firings_out.mda --adjacency_radius=-1 --detect_sign=-1 --detect_threshold=3 --_tempdir=/media/anna/whirlwindtemp2/MSjstmp/tempdir_9cacaa7f0f_IqVfQp
Using tempdir=/media/anna/whirlwindtemp2/MSjstmp/tempdir_9cacaa7f0f_IqVfQp
Preparing /media/anna/whirlwindtemp2/MSjstmp/tempdir_9cacaa7f0f_IqVfQp/timeseries.hdf5...
Preparing neighborhood sorters...

Detecting events on channel 4 (phase1

RUNNING: ml-run-process pyms.anneal_segments --inputs firings_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/firings-1.mda firings_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/firings-2.mda firings_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/firings-3.mda timeseries_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre-1.mda timeseries_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre-2.mda timeseries_list:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre-3.mda --parameters time_offsets:[0, 47461993, 156381992] --outputs dmatrix_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_dmatrix_out_8e2bf89bdddd34e16eb6c9d045a6af1c0319e7c2.prv dmatrix_templates_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_dmatrix_tem

RUNNING: ml-run-process ms3.combine_cluster_metrics --inputs metrics_list:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_3214901bfcceb12f76683f54e8b1f7c311c5fcac.prv metrics_list:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_metrics_out_7c0c32ce10c5c0bd6577642b208475a7aac7c85d.prv --parameters --outputs metrics_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/metrics_raw.json
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_3214901bfcceb12f76683f54e8b1f7c311c5fcac.prv ...
Locating /media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_metrics_out_7c0c32ce10c5c0bd6577642b208475a7aac7c85d.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs.

In [ ]:
pdb

In [ ]:
# in a separate loop, go back and add tags to the metrics - due to bug in ms3.combine_cluster_metrics
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.add_curation_tags(dataset_dir=nt_dir,output_dir=nt_dir,opts={})


In [ ]:
# generate marks 
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.extract_marks(dataset_dir=nt_dir,output_dir=nt_dir,opts={})

In [ ]:
# TODO: implement logging system to track all calls and params used 